In [1]:
# suppress warnings
import warnings
warnings.simplefilter('ignore')

#import packages
import numpy as np
from glob import glob
import pandas as pd
from matplotlib import pyplot as plt
import subprocess
from datetime import datetime
from plotnine import *


#local imports from opensoundscape
from opensoundscape.audio import Audio
from opensoundscape.spectrogram import Spectrogram
from opensoundscape.ribbit import ribbit

# create big visuals
plt.rcParams['figure.figsize']=[15,8]
pd.set_option('display.precision', 2)

In [18]:
# list of file paths where ribbit scores are saved. 
# If the model was run once with all of the data, this will be a single file path. 
# If the data was run in several chunks simultaneously, this will be a file path for each set of data
rs_fp = ["./ribbit_scores_flshe2017/ribbit_scores.csv"]

sa_flshe_df = pd.read_csv(rs_fp, index_col = 0) # read in ribbit scores
sa_flshe_df['date'] = pd.to_datetime(sa_flshe_df.index.str[-19:-4], format='%Y%m%d_%H%M%S', errors='coerce') #extract date from file path
sa_flshe_df
#sa_pulse_df['date']=pd.to_datetime(sa_pulse_df['date'])


,score,year,logger,time_stamp,date
/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_400/TEST_0+1_20170213_212100.wav,546.87,2017,400,286.0,2017-02-13 21:21:00
/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_400/TEST_0+1_20170213_232100.wav,450.13,2017,400,273.0,2017-02-13 23:21:00
/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_400/TEST_0+1_20170211_231900.wav,442.50,2017,400,61.5,2017-02-11 23:19:00
/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_400/TEST_0+1_20170213_202100.wav,412.17,2017,400,245.5,2017-02-13 20:21:00
/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_400/TEST_0+1_20170212_212000.wav,388.02,2017,400,49.5,2017-02-12 21:20:00
...,...,...,...,...,...
/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_401/TEST_0+1_20170223_003358.wav,10.10,2017,401,0.5,2017-02-23 00:33:58
/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_398/TEST_0+1_20170619_205158.wav,9.58,2017,398,0.0,2017-06-19 20:51:58
/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_401/TEST_0+1_20170127_211058.wav,9.34,2017,401,0.0,2017-01-27 21:10:58
/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_402/FLSH-402_0+1_20170223_003358.wav,9.11,2017,402,0.0,2017-02-23 00:33:58


In [10]:
sa_flshe_df.index

Index(['/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_398/TEST_0+1_20161031_174700.wav',
       '/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_398/TEST_0+1_20161031_184700.wav',
       '/Volumes/Expansion/Frog Call Project/Calling Data/FLSHE/FLSHE_2017/FLSHE_2017_398/TEST_0+1_20170104_174500.wav'],
      dtype='object')